In [2]:
import pandas as pd
import numpy as np

In [3]:
# Load data from CSV
df = pd.read_csv("../data/Building_Violations_sample_50000.csv")


/Users/michael/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1159: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [4]:
## Clean data

# Delete useless variable
del df['Unnamed: 0']

# Convert column headers to snake case
import re

def spaces_to_snake(str_with_spaces):
    s = re.sub('\s+','_',str_with_spaces)
    return s.lower()

df.columns = [spaces_to_snake(col) for col in df.columns]
df.columns

# Delete SSA column
del df['ssa']

# Convert dates to datetime format
df.violation_date = pd.to_datetime(df.violation_date)
df.violation_status_date = pd.to_datetime(df.violation_status_date)
df.violation_last_modified_date = pd.to_datetime(df.violation_last_modified_date)

In [6]:
# Clean missing values
df.violation_description = df.violation_description.fillna('Empty description')

In [7]:
df.violation_ordinance = df.violation_ordinance.fillna('Missing ordinance')

In [45]:
# Count missing values
df.isnull().sum(axis=0)

id                                  0
violation_last_modified_date        0
violation_date                      0
violation_code                      0
violation_status                    0
violation_status_date           36488
violation_description               0
violation_location              32484
violation_inspector_comments     5295
violation_ordinance                 0
inspector_id                        0
inspection_number                   0
inspection_status                   0
inspection_waived                   0
inspection_category                 0
department_bureau                   0
address                             0
property_group                      0
latitude                           48
longitude                          48
location                           48
dtype: int64

In [8]:
# Write CSV with cleaned data
df.to_csv("../data/Building_Violations_sample_50000_clean.csv")

In [9]:
df.dtypes

id                                       int64
violation_last_modified_date    datetime64[ns]
violation_date                  datetime64[ns]
violation_code                          object
violation_status                        object
violation_status_date           datetime64[ns]
violation_description                   object
violation_location                      object
violation_inspector_comments            object
violation_ordinance                     object
inspector_id                            object
inspection_number                        int64
inspection_status                       object
inspection_waived                       object
inspection_category                     object
department_bureau                       object
address                                 object
property_group                           int64
latitude                               float64
longitude                              float64
location                                object
dtype: object

In [10]:
## Explore violation codes ##

# Check how many different violation codes there are
df.violation_code.describe()

count        50000
unique         757
top       CN190019
freq          2463
Name: violation_code, dtype: object

In [36]:
df.violation_description.describe()

count                          50000
unique                           725
top       ARRANGE PREMISE INSPECTION
freq                            2463
Name: violation_description, dtype: object

In [37]:
df.violation_ordinance.describe()

count                                               50000
unique                                                717
top       Arrange for inspection of premises. (13-12-100)
freq                                                 2463
Name: violation_ordinance, dtype: object

In [11]:
# List violation descriptions
df.violation_description.value_counts()

ARRANGE PREMISE INSPECTION        2463
POST OWNER/MANAGERS NAME/#        1619
REPAIR EXTERIOR WALL              1488
REPAIR PORCH SYSTEM               1326
VACANT BUILDING - REGISTER        1065
REPLCE WINDOW PANES, PLEXGLAS     1011
PLANS & PERMITS REQ - CONTRCTR     964
REPAIR EXTERIOR STAIR              937
FILE BLDG REGISTRATION             904
INSTALL SMOKE DETECTORS            892
MAINTAIN OR REPAIR ELECT ELEVA     730
DEBRIS, EXCESSIVE                  725
STOP/REMOVE NUISANCE               696
CARB MONOX DETECT IN RESID         638
OBSTRUCTIONS IN EXIT WAY           625
...
CARB MONOX DETECT IN HOTEL        1
ELIMINATE TRIPPING HAZARD PASS    1
ENCL/SECURE OPENINGS PASS         1
REP/REPL HANDRAIL FOR ESCL        1
IDENTIFY ELE MACHINES             1
PRV CONDENSER                     1
REP/REPL DEF HALL BUTTONS PASS    1
HEATER 8FT ABOVE FLR IN GARAGE    1
PROVIDE HOT WATER                 1
PRV ABSORBER                      1
VERIFICATIONS OF D.U.             1
NONCOMPLIANT MA

In [32]:
# Confirm that there is a m:1 mapping from violation_DESCRIPTION to violation_code
grp_code = df.groupby(['violation_code'])
grp_desc = df.groupby(['violation_description'])
grp_code_desc = df.groupby(['violation_code', 'violation_description'])

In [33]:
grp_code.id.count().shape

(757,)

In [34]:
grp_desc.id.count().shape

(725,)

In [35]:
grp_code_desc.id.count().shape

(757,)

In [25]:
# Confirm that there is a m:1 mapping from violation_ORDINANCE to violation_code
grp_ord = df.groupby(['violation_ordinance'])
grp_code_ord = df.groupby(['violation_code', 'violation_ordinance'])

In [26]:
grp_ord.id.count().shape

(717,)

In [27]:
grp_code_ord.id.count().shape

(757,)

In [30]:
# Create a list of violation codes (unique key) mapped to their descriptions and ordinances
list_of_violations = df[['violation_code', 'violation_description', 'violation_ordinance']].sort(['violation_code']).drop_duplicates()
list_of_violations.to_csv('../data documentation/list_of_violations.csv',index=False)

In [31]:
list_of_violations.head()

,violation_code,violation_description,violation_ordinance
14807,020008,Empty description,Missing ordinance
48248,190019,NO ENTRY,Missing ordinance
25480,199029,OPEN TYPE VIOLATION,Missing ordinance
11253,BR1000,LICENSED CONTRACTOR,The code violations listed below must be corre...
33151,BR1001,OWNER OR LICENSED CONTRACTOR,The code violations listed below must be corre...


In [14]:
## Explore violation status ##

# Check how many kinds of violation status
df.violation_status.describe()

count     50000
unique        3
top        OPEN
freq      36355
Name: violation_status, dtype: object

In [15]:
df.violation_status.value_counts()

OPEN        36355
COMPLIED    13500
NO ENTRY      145
dtype: int64

In [16]:
# Violation location
# (an ugly string variable)
df.violation_location.head(100)

0                    OTHER   :    :
1     OTHER   :    :MULTIPLE FLOORS
2                    OTHER   :    :
3                    OTHER   :    :
4                    OTHER   :    :
5                    OTHER   :    :
6                    OTHER   :    :
7                    EXTERIOR:W   :
8                    OTHER   :    :
9            OTHER   :    :BUILDING
10                   OTHER   :    :
11                   OTHER   :    :
12                   EXTERIOR:S   :
13                              NaN
14                   EXTERIOR:W   :
...
85            OTHER   :    :
86            OTHER   :    :
87                       NaN
88            OTHER   :    :
89            OTHER   :    :
90            OTHER   :    :
91            INTERIOR:-1  :
92            OTHER   :    :
93                       NaN
94            OTHER   :    :
95                       NaN
96            EXTERIOR:S   :
97    OTHER   :    :BUILDING
98                       NaN
99                       NaN
Name: violation_locat

In [43]:
df.groupby(['violation_location']).size()

violation_location
#100300460                         1
#127 COMPLIED                      1
#127 TONY TINERELLA COMPLIED       1
#2 CLEAVER BROOKS OK 7/10/06       1
#208 COMPLIED                      1
#2ND FLOOR HOT WATER TEMP 102F     1
#507  #510                         1
#BL1001                            2
-                                  1
//112464                           1
//BL00748                          1
//BL00804                          1
//BL00946                         61
//BL00946, PLAN OBTAINED           1
//BL00946, SEASONAL                1
...
WEST ELEVATION  -                  1
WEST ELEVATIONS -                  1
WEST YARD                          2
WEST, SOUTH , NORTH                1
WEST, STAIRWAY                     1
WHOLE HOUSE                        1
WINDOW                             1
WORK DONE BY ANDEE BOILER # 786    1
WORK DONE BY ANDEE BOILER #462     1
X                                  2
XX                                 8
XX:INSPECTION  

In [38]:
df.violation_inspector_comments.describe()

count                         44705
unique                        40583
top       NO OWNER I.D SIGN POSTED.
freq                             72
Name: violation_inspector_comments, dtype: object

In [46]:
## Inspector Info ##

df.inspector_id.describe()

count       50000
unique        273
top       BL00444
freq         2180
Name: inspector_id, dtype: object

In [47]:
df.inspector_id.head()

0    BL00444
1    BL00748
2    BL00746
3    BL00746
4     104976
Name: inspector_id, dtype: object

count       50000.000000
mean      6195288.986320
std       4296079.998463
min        375113.000000
25%       2021217.500000
50%       2813561.000000
75%      10630848.750000
max      11597625.000000
Name: inspection_number, dtype: float64